In [ ]:
%load_ext autoreload
%autoreload 2

import safep
from glob import glob
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [ ]:
logfile = 'RFEP_decouple.log'

In [ ]:
global_conf, colvars, biases, TI_traj = safep.parse_Colvars_log(logfile)
for c in colvars:
    print(c['name'])
for b in biases:
    print(b['name'])

# Need to pick the right restraint if there are several
# TODO look for moving harmonic wall, there should not be many - this can be done for each
restraint_config = biases[0]
rest_name = restraint_config['name']

print(f'Processing TI data for restraint {rest_name}')


path = os.path.dirname(logfile) # We assume the colvars traj and log are in the same directory

if 'traj_file' in global_conf:
    colvarsTraj = path + global_conf['traj_file']
else:
    colvarsTraj = path + global_conf['output_prefix'] + '.colvars.traj'

In [ ]:
# Extract precomputed TI info from traj file
dAdL = TI_traj[rest_name]['dAdL']
lambdas =  TI_traj[rest_name]['L']
dL = lambdas[1] - lambdas[0] # FIXME assumes uniform spacing
if restraint_config['decoupling']: # lambdas are in reverse order
    dAdL = dAdL[::-1]
    lambdas = lambdas[::-1]

In [ ]:
# Setup and processing of colvars data
with open(colvarsTraj) as f:
    first_line = f.readline()
columns = re.split(' +', first_line)[1:-1]
dataTI = pd.read_csv(colvarsTraj, delim_whitespace=True, names=columns, comment='#', index_col=0)

# Process the harmonic wall restraint
DBC_rest = safep.make_harmonicWall_from_Colvars(restraint_config)
nLambdas = DBC_rest['numStages'] + 1
n_equil = DBC_rest['targetEQ']
cv =  DBC_rest['colvar']
dataTI = dataTI.rename(columns={cv:'DBC'})

if DBC_rest['schedule'] == []:
    schedule = np.linspace(0, 1, nLambdas) 
else:
    schedule = DBC_rest['schedule']

dataTI = dataTI[dataTI.index>=n_equil][1:] # Remove first samples of each window from analysis
dataTI.index = dataTI.index-n_equil

Ls = (dataTI.index.values-1)//DBC_rest['numSteps']
Ls[0] = 0
Ls[Ls==41] = 40 # This is a small hack in case there are extra samples for the last window

dataLs = np.round([schedule[i] for i in Ls], 3)
dataTI.loc[:,'L'] = dataLs
dataTI = dataTI.iloc[1:]

TIperWindow, TIcumulative = safep.process_TI(dataTI, DBC_rest, None)

In [ ]:
dG_DBC = np.round(TIcumulative['dG'][1], 1)
error_DBC = np.round(TIcumulative['error'][1], 1)
change_mkd_TI = f'\u0394G<sub>DBC</sub> = {dG_DBC} kcal/mol'
error_mkd_TI = f'Standard Deviation: {error_DBC} kcal/mol'
print(change_mkd_TI, error_mkd_TI)

In [ ]:
''' Plot the results. '''
fig, axes = safep.plot_TI(TIcumulative, TIperWindow, fontsize=20)
axes[1].plot(1-np.array(lambdas), np.array(dAdL)*dL, marker='o', label='Colvars internal dAdlambda', color='red')
axes[1].legend()
plt.show()